In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from RecursiveFNO import RecursiveFNO
from glob import glob
from torch.utils.data import TensorDataset, DataLoader
from neuralop.models.base_model import get_model
from configmypy import ConfigPipeline, YamlConfig, ArgparseConfig


os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.manual_seed(66)
np.random.seed(66)
torch.set_default_dtype(torch.float32)
BATCH_SIZE = 1
SEQ_LEN = 15

In [2]:
def load_checkpoint(model, optimizer, scheduler, save_dir):
    '''load model and optimizer'''
    checkpoint = torch.load(save_dir)
    model.load_state_dict(checkpoint['model_state_dict'])
    if (not optimizer is None):
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    print('Pretrained model loaded!')
    return model, optimizer, scheduler

In [3]:
config_name = "default"
pipe = ConfigPipeline(
    [
        YamlConfig("em_pino_config.yaml", config_name="default", config_folder="config"),
    ]
)
config = pipe.read_conf()
config_name = pipe.steps[-1].config_name
model_fno = get_model(config).cuda()
model = RecursiveFNO(model_fno, SEQ_LEN-1).cuda()
model.eval()
model, _, _ = load_checkpoint(model, None, None, "./model/checkpoint1000.pt")

Pretrained model loaded!


In [4]:
from EnergeticMatDataPipeLine import EnergeticMatDataPipeLine


min_state = [300.0, -1184690200.0, 0.0]
max_state = [5000.0, 79642935000.0, 1.0]
min_vel = [-1704.224, -4531.192]
max_vel = [9019.604, 6652.869]
testing_state, testing_vel = EnergeticMatDataPipeLine.clip_raw_data((150, 200), sequence_length=SEQ_LEN, purpose="")
testing_state_norm = EnergeticMatDataPipeLine.data_normalization_test(testing_state, min_state, max_state, 3)
testing_vel_norm = EnergeticMatDataPipeLine.data_normalization_test(testing_vel, min_vel, max_vel, 2)

In [5]:
b, x, y, _ = testing_state_norm.shape
testing_state_vel_norm = np.empty((b, x, y, SEQ_LEN * 5))
for i in range(SEQ_LEN):
    testing_state_vel_norm[:, :, :, i*5:i*5+3] = testing_state_norm[:, :, :, i*3:(i+1)*3]
    testing_state_vel_norm[:, :, :, i*5+3:i*5+5] = testing_vel_norm[:, :, :, i*2:(i+1)*2]
testing_state_vel_norm = np.transpose(testing_state_vel_norm, (0, 3, 2, 1))

In [6]:
test_ic = torch.Tensor(testing_state_vel_norm[:, :5, :, :])
test_seq = torch.Tensor(testing_state_vel_norm[:, 5:, :, :])
test_dataset = TensorDataset(test_ic, test_seq)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
test_case = []
for i in range(150, 200):
    file_path = os.path.join(os.sep,'scratch','xc7ts','fno', 'em', 'single_void_data', f'void_{i}.npy')
    if os.path.exists(file_path):
        test_case.append(i)

In [8]:
len(test_case)

34

In [9]:
with torch.no_grad():
    model.eval()
    for batch_id, data in enumerate(test_dataloader):
        ic, seq = data
        ic, seq = ic.cuda(), seq.cuda()
        output = model(ic)
        output = output[0].transpose(0, 2).detach().cpu().numpy()
        fname = "pred_fno/void_%i.npy" % (test_case[batch_id])
        np.save(fname, output)

In [10]:
output.shape

(128, 256, 70)